In [1]:
!pip install -U openmim
!mim install mmcv-full

     |████████████████████████████████| 50 kB 977 kB/s eta 0:00:01
     |████████████████████████████████| 237 kB 2.3 MB/s eta 0:00:01
     |████████████████████████████████| 51 kB 3.7 MB/s  eta 0:00:01
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.7.4.3
    Uninstalling typing-extensions-3.7.4.3:
      Successfully uninstalled typing-extensions-3.7.4.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.4.1 requires typing-extensions~=3.7.4, but you have typing-extensions 4.4.0 which is incompatible.
bokeh 2.2.3 requires tornado>=5.1, but you have tornado 5.0.2 which is incompatible.
arviz 0.11.1 requires typing-extensions<4,>=3.7.4.3, but you have typing-extensions 4.4.0 which is incompatible.
aiobotocore 1.2.0 requires botocore<1.19.53,>=1.19.52, but you have botocore 1.20.5 which is incompatible.
You

In [2]:
!git clone https://github.com/open-mmlab/mmcv.git
%cd /kaggle/working/mmcv
!pip install -r requirements.txt

Cloning into 'mmcv'...
remote: Enumerating objects: 13712, done.
remote: Counting objects: 100% (615/615), done.
remote: Compressing objects: 100% (415/415), done.
remote: Total 13712 (delta 330), reused 394 (delta 195), pack-reused 13097
Receiving objects: 100% (13712/13712), 13.05 MiB | 29.05 MiB/s, done.
Resolving deltas: 100% (9825/9825), done.
/kaggle/working/mmcv
Ignoring regex: markers 'sys_platform == "win32"' don't match your environment
     |████████████████████████████████| 120 kB 2.0 MB/s eta 0:00:01
     |████████████████████████████████| 210 kB 12.5 MB/s eta 0:00:01
     |████████████████████████████████| 602 kB 34.3 MB/s eta 0:00:01
     |████████████████████████████████| 4.5 MB 55.0 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 50.7 MB/s eta 0:00:01
     |████████████████████████████████| 46 kB 2.6 MB/s  eta 0:00:01
     |████████████████████████████████| 15.7 MB 45.1 MB/s eta 0:00:01
     |████████████████████████████████| 86 kB 3.7 MB/s  eta 0:00:01

In [3]:
%cd /kaggle/working
!git clone https://github.com/open-mmlab/mmsegmentation.git
%cd /kaggle/working/mmsegmentation
!pip install -v -e .

/kaggle/working
Cloning into 'mmsegmentation'...
remote: Enumerating objects: 12050, done.
remote: Counting objects: 100% (161/161), done.
remote: Compressing objects: 100% (121/121), done.
remote: Total 12050 (delta 72), reused 94 (delta 39), pack-reused 11889
Receiving objects: 100% (12050/12050), 14.85 MiB | 33.05 MiB/s, done.
Resolving deltas: 100% (8780/8780), done.
/kaggle/working/mmsegmentation
Using pip 21.0.1 from /opt/conda/lib/python3.7/site-packages/pip (python 3.7)
Non-user install because site-packages writeable
Created temporary directory: /tmp/pip-ephem-wheel-cache-r4oihrfp
Created temporary directory: /tmp/pip-req-tracker-yjj45vxy
Initialized build tracking at /tmp/pip-req-tracker-yjj45vxy
Created build tracker: /tmp/pip-req-tracker-yjj45vxy
Entered build tracker: /tmp/pip-req-tracker-yjj45vxy
Created temporary directory: /tmp/pip-install-94b8dy63
Obtaining file:///kaggle/working/mmsegmentation
  Added file:///kaggle/working/mmsegmentation to build tracker '/tmp/pip-re

In [4]:
%%writefile configs/_base_/models/danet_r50-d8.py
norm_cfg = dict(type='BN', requires_grad=True)
model = dict(
    type='EncoderDecoder',
    pretrained='open-mmlab://resnet50_v1c',
    backbone=dict(
        type='ResNetV1c',
        depth=50,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        dilations=(1, 1, 2, 4),
        strides=(1, 2, 1, 1),
        norm_cfg=norm_cfg,
        norm_eval=False,
        style='pytorch',
        contract_dilation=True),
    decode_head=dict(
        type='ASPPHead',
        in_channels=2048,
        in_index=3,
        channels=512,
        dilations=(1, 12, 24, 36),
        dropout_ratio=0.1,
        num_classes=2,
        norm_cfg=norm_cfg,
        align_corners=False,
        loss_decode=dict(
            type='CrossEntropyLoss', use_sigmoid=False, loss_weight=1.0)),
    auxiliary_head=dict(
        type='FCNHead',
        in_channels=1024,
        in_index=2,
        channels=256,
        num_convs=1,
        concat_input=False,
        dropout_ratio=0.1,
        num_classes=2,
        norm_cfg=norm_cfg,
        align_corners=False,
        loss_decode=dict(
            type='CrossEntropyLoss', use_sigmoid=False, loss_weight=0.4)),
    # model training and testing settings
    train_cfg=dict(),
    test_cfg=dict(mode='whole'))

Overwriting configs/_base_/models/danet_r50-d8.py


In [5]:
%%writefile mmseg/datasets/my_dataset.py
import mmcv
from mmcv.utils import print_log

from ..utils import get_root_logger
from .builder import DATASETS
from .custom import CustomDataset
import os.path as osp

@DATASETS.register_module()

class Mydataset(CustomDataset):
   
    CLASSES = ('background','mri')

    PALETTE = [[0,0,0],[255,255,255]]


    def __init__(self, **kwargs):
        super(Mydataset, self).__init__(
            img_suffix='.png',
            seg_map_suffix='.png',
            reduce_zero_label=False,**kwargs)
        assert osp.exists(self.img_dir)

   

Writing mmseg/datasets/my_dataset.py


In [6]:
%%writefile configs/_base_/datasets/my_dataset.py
dataset_type = 'Mydataset' #上一步中你定义的数据集的名字
data_root = '/kaggle/input/mri-for-mmsegmentation' #数据集存储路径
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
"""
This crop_size setting is followed by the implementation of
`PointFlow: Flowing Semantics Through Points for Aerial Image
Segmentation <https://arxiv.org/pdf/2103.06564.pdf>`_.
"""

crop_size = (512,512)

train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations'),
    dict(type='Resize', img_scale=(896, 896), ratio_range=(0.5, 2.0)),
    dict(type='RandomCrop', crop_size=crop_size, cat_max_ratio=0.75),
    dict(type='RandomFlip', prob=0.5),
    dict(type='PhotoMetricDistortion'),
    dict(type='Normalize', **img_norm_cfg),
    dict(type='Pad', size=crop_size, pad_val=0, seg_pad_val=255),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_semantic_seg']),
]
test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(512,512),
        # img_ratios=[0.5, 0.75, 1.0, 1.25, 1.5, 1.75],
        flip=False,
        transforms=[
            dict(type='Resize', keep_ratio=True),
            dict(type='RandomFlip'),
            dict(type='Normalize', **img_norm_cfg),
            dict(type='ImageToTensor', keys=['img']),
            dict(type='Collect', keys=['img']),
        ])
]
data = dict(
    samples_per_gpu=8,
    workers_per_gpu=4,
    train=dict(
        type='RepeatDataset',
        times=1,
        dataset=dict(
            type=dataset_type,
            data_root=data_root,
            img_dir='images/train',
            ann_dir='labels/train',
            pipeline=train_pipeline)),
    val=dict(
        type=dataset_type,
        data_root=data_root,
        img_dir='images/val',
        ann_dir='labels/val',
        pipeline=test_pipeline),
    test=dict(
        type=dataset_type,
        data_root=data_root,
        img_dir='images/val',
        ann_dir='labels/val',
        pipeline=test_pipeline))

Writing configs/_base_/datasets/my_dataset.py


In [7]:
%%writefile mmseg/datasets/__init__.py
from .ade import ADE20KDataset
from .builder import DATASETS, PIPELINES, build_dataloader, build_dataset
from .chase_db1 import ChaseDB1Dataset
from .cityscapes import CityscapesDataset
from .coco_stuff import COCOStuffDataset
from .custom import CustomDataset
from .dark_zurich import DarkZurichDataset
from .dataset_wrappers import (ConcatDataset, MultiImageMixDataset,
                               RepeatDataset)
from .drive import DRIVEDataset
from .hrf import HRFDataset
from .isaid import iSAIDDataset
from .isprs import ISPRSDataset
from .loveda import LoveDADataset
from .night_driving import NightDrivingDataset
from .pascal_context import PascalContextDataset, PascalContextDataset59
from .potsdam import PotsdamDataset
from .stare import STAREDataset
from .voc import PascalVOCDataset
import sys
sys.path.append('..')
from .my_dataset import Mydataset
__all__ = [
    'CustomDataset', 'build_dataloader', 'ConcatDataset', 'RepeatDataset',
    'DATASETS', 'build_dataset', 'PIPELINES', 'CityscapesDataset',
    'PascalVOCDataset', 'ADE20KDataset', 'PascalContextDataset',
    'PascalContextDataset59', 'ChaseDB1Dataset', 'DRIVEDataset', 'HRFDataset',
    'STAREDataset', 'DarkZurichDataset', 'NightDrivingDataset',
    'COCOStuffDataset', 'LoveDADataset', 'MultiImageMixDataset',
    'iSAIDDataset', 'ISPRSDataset', 'PotsdamDataset','Mydataset'
]

Overwriting mmseg/datasets/__init__.py


In [8]:
%%writefile configs/_base_/schedules/schedule_20k.py
optimizer = dict(type='SGD', lr=0.01, momentum=0.9, weight_decay=0.0005)
optimizer_config = dict()
# learning policy
lr_config = dict(policy='poly', power=0.9, min_lr=1e-4, by_epoch=False)
runner = dict(type='EpochBasedRunner', max_epochs=20) #按epoch的方式进行迭代
checkpoint_config = dict(by_epoch=True, interval=5) #每多少次迭代保存一次模型
evaluation = dict(interval=1 ,metric='mIoU')  # 每多少次迭代计算一次指标

Overwriting configs/_base_/schedules/schedule_20k.py


In [9]:
%%writefile configs/pspnet/pspnet_r50-d8_512x512_160k_ade20k.py 
_base_ = [
    '../_base_/models/danet_r50-d8.py',
    '../_base_/datasets/my_dataset.py', '../_base_/default_runtime.py',
    '../_base_/schedules/schedule_20k.py'
]
model = dict(
    decode_head=dict(num_classes=2), auxiliary_head=dict(num_classes=2))

Overwriting configs/pspnet/pspnet_r50-d8_512x512_160k_ade20k.py


In [10]:
!nvidia-smi

Thu Oct 27 07:06:42 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla T4            Off  | 00000000:00:05.0 Off |                    0 |
| N/A   

In [ ]:
!python tools/train.py --gpu-id 0 --gpu-id 1 configs/pspnet/pspnet_r50-d8_512x512_160k_ade20k.py 

2022-10-27 07:21:08,219 - mmseg - INFO - Multi-processing start method is `None`
2022-10-27 07:21:08,219 - mmseg - INFO - OpenCV num_threads is `2
2022-10-27 07:21:08,247 - mmseg - INFO - Environment info:
------------------------------------------------------------
sys.platform: linux
Python: 3.7.9 | packaged by conda-forge | (default, Dec  9 2020, 21:08:20) [GCC 9.3.0]
CUDA available: True
GPU 0,1: Tesla T4
CUDA_HOME: /usr/local/cuda
NVCC: Cuda compilation tools, release 10.2, V10.2.8
GCC: gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
PyTorch: 1.7.0
PyTorch compiling details: PyTorch built with:
  - GCC 7.3
  - C++ Version: 201402
  - Intel(R) Math Kernel Library Version 2020.0.2 Product Build 20200624 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v1.6.0 (Git Hash 5ef631a030a6f73131c77892041042805a06064f)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 10.2
  - NVCC architecture flags: -gencode;arch=compute_37,cod

In [13]:
!python tools/test.py configs/pspnet/pspnet_r50-d8_512x512_160k_ade20k.py work_dirs/pspnet_r50-d8_512x512_160k_ade20k/latest.pth --eval mIoU --show-dir out

2022-10-18 10:17:13,214 - mmseg - INFO - Multi-processing start method is `None`
2022-10-18 10:17:13,214 - mmseg - INFO - OpenCV num_threads is `2
2022-10-18 10:17:13,229 - mmseg - INFO - Loaded 600 images
/kaggle/working/mmsegmentation/mmseg/models/decode_heads/decode_head.py:94: UserWarning: For binary segmentation, we suggest using`out_channels = 1` to define the outputchannels of segmentor, and use `threshold`to convert seg_logist into a predictionapplying a threshold
  warnings.warn('For binary segmentation, we suggest using'
/kaggle/working/mmsegmentation/mmseg/models/losses/cross_entropy_loss.py:236: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  'Default ``avg_non_ignore`` is False, if you would like to '
load checkpoint from local path: work_dirs/pspnet_r50-d8_512x512_160k_ade20k/latest.pth
tools/test.py

In [15]:
!tar -cvf /kaggle/working/et.tar.gz out

out/
out/2604.png
out/1979.png
out/2436.png
out/110.png
out/201.png
out/1182.png
out/955.png
out/1896.png
out/2951.png
out/1144.png
out/1659.png
out/599.png
out/1351.png
out/2234.png
out/2484.png
out/2860.png
out/1324.png
out/2719.png
out/1285.png
out/1838.png
out/2552.png
out/1738.png
out/980.png
out/2686.png
out/1681.png
out/702.png
out/2597.png
out/116.png
out/2030.png
out/892.png
out/1630.png
out/1809.png
out/380.png
out/600.png
out/724.png
out/2345.png
out/64.png
out/902.png
out/1006.png
out/1463.png
out/264.png
out/621.png
out/1747.png
out/1880.png
out/317.png
out/1001.png
out/982.png
out/1687.png
out/2147.png
out/1952.png
out/274.png
out/1812.png
out/1058.png
out/2265.png
out/2110.png
out/2031.png
out/2958.png
out/1852.png
out/2004.png
out/1510.png
out/1377.png
out/1279.png
out/2495.png
out/993.png
out/2588.png
out/2852.png
out/27.png
out/1627.png
out/1634.png
out/2657.png
out/2865.png
out/3022.png
out/1432.png
out/720.png
out/1891.png
out/2802.png
out/1215.png
out/1728.png
out/